In [ ]:
# https://adventofcode.com/2018/day/16

struct Example
    before::Array{Int}
    instruction::Array{Int}
    after::Array{Int}
    
    function Example(lines::Array{String})
        line1 = match(r"Before: \[(\d+), (\d+), (\d+), (\d+)\]", lines[1]).captures
        line2 = match(r"(\d+) (\d+) (\d+) (\d+)", lines[2]).captures
        line3 = match(r"After:  \[(\d+), (\d+), (\d+), (\d+)\]", lines[3]).captures
        new(parse.(Int, line1), parse.(Int, line2), parse.(Int, line3))
    end
end

examples, program = open("16.txt", "r") do file
    examples = []
    while true
        line1 = readline(file)
        if line1 == "" break end
        lines = [readline(file) for _ in 1:3]
        examples = [examples; Example([[line1]; lines])]
    end
    
    readline(file)
    
    program = readlines(file) .|> split .|> l -> parse.(Int, l)
    
    examples, program
end


In [7]:
addr(r, a, b, c) = (r = copy(r); r[c + 1] = r[a + 1] + r[b + 1]; r)
addi(r, a, b, c) = (r = copy(r); r[c + 1] = r[a + 1] + b; r)
mulr(r, a, b, c) = (r = copy(r); r[c + 1] = r[a + 1] * r[b + 1]; r)
muli(r, a, b, c) = (r = copy(r); r[c + 1] = r[a + 1] * b; r)
banr(r, a, b, c) = (r = copy(r); r[c + 1] = r[a + 1] & r[b + 1]; r)
bani(r, a, b, c) = (r = copy(r); r[c + 1] = r[a + 1] & b; r)
borr(r, a, b, c) = (r = copy(r); r[c + 1] = r[a + 1] | r[b + 1]; r)
bori(r, a, b, c) = (r = copy(r); r[c + 1] = r[a + 1] | b; r)
setr(r, a, b, c) = (r = copy(r); r[c + 1] = r[a + 1]; r)
seti(r, a, b, c) = (r = copy(r); r[c + 1] = a; r)
gtir(r, a, b, c) = (r = copy(r); r[c + 1] = a > r[b + 1]; r)
gtri(r, a, b, c) = (r = copy(r); r[c + 1] = r[a + 1] > b; r)
gtrr(r, a, b, c) = (r = copy(r); r[c + 1] = r[a + 1] > r[b + 1]; r)
eqir(r, a, b, c) = (r = copy(r); r[c + 1] = a == r[b + 1]; r)
eqri(r, a, b, c) = (r = copy(r); r[c + 1] = r[a + 1] == b; r)
eqrr(r, a, b, c) = (r = copy(r); r[c + 1] = r[a + 1] == r[b + 1]; r)

ops = [
    addr, addi,
    mulr, muli,
    banr, bani,
    borr, bori,
    setr, seti,
    gtir, gtri, gtrr,
    eqir, eqri, eqrr,
]


16-element Array{Function,1}:
 addr
 addi
 mulr
 muli
 banr
 bani
 borr
 bori
 setr
 seti
 gtir
 gtri
 gtrr
 eqir
 eqri
 eqrr

In [8]:
# part 1

isvalid(e, o) = o(e.before, e.instruction[2:end]...) == e.after

valid = [length([o for o in ops if isvalid(e, o)]) for e in examples]

valid[valid .>= 3] |> length


570

In [9]:
# part 2

valid = [copy(ops) for i in 1:length(ops)]

# apply example constraints
for example in examples
    i = example.instruction[1] + 1
    valid[i] = [v for v in valid[i] if isvalid(example, v)]
end

# reduce the valid sets based on exact matches
while any(length.(valid) .> 1)
    for i in 1:length(valid)
        if length(valid[i]) == 1
            done = valid[i][1]
            for j in 1:length(valid)
                if j != i && done in valid[j]
                    valid[j] = [v for v in valid[j] if v != done]
                end
            end
        end
    end
end
                                            
# run the example program
valid = [v[1] for v in valid]
registers = [0, 0, 0, 0]
for instruction in program
    registers = valid[instruction[1] + 1](registers, instruction[2:end]...)
end

registers[1]


503